# Projekt 2 - Location Intelligence - podsumowanie

## Grupa: Klaudia Buksa, Grzegorz Cichy, Marcin Słyś

## **Etapy wykonane w trakcie realizacji projektu**

### 1. Wczytanie danych

Pierwszym etapem wykonania projektu było wczytanie danych wejściowych. Były to pliki .parquet zawierające informacje o ścieżkach rowerowych w 2 miastach - Amsterdamie i Krakowie.


Po wczytaniu plików jako DataFrame zawierały one kolumnę 'geometry' przechowującą LINESTRING oraz wiele innych kolumn zawierających w przeważającej części wartości brakujące.
Wszystkie kolumny poza geometrią zostały na późniejszym etapie odrzucone, a dane w celu wykorzystania kontekstu przestrzennego zostały przetransformowane do GeoDataFrame.

### 2. Utworzenie Siatek heksagonalnych H3.

Dla obszarów miast utworzone zostały heksagonalne siatki, które na późniejszym etapie posłużyć miały do agregacji danych. Zostały one przycięte do granicy obszaru miejskiego, aby uniknąć pustych hexów w danych - jeśli siatka nie znajdowała się w granicach Amsterdamu to nie powinna być uwzględniana.

Utworzona siatka po przycięciu oraz z nałożonymi na nią ścieżkami rowerowymi oraz granicami Amsterdamu:

!['Siatka'](./images/grid_bike_trails.png)

### 3. Inżynieria cech.

**3.1 Zmienna objaśniana**

Po nałożeniu siatek na trasy ścieżek rowerowych można było przejść do agregacji na poziomie heksagonu w celu utworzenia cech mających posłużyć do późniejszej predykcji.

W pierwszej kolejności wyznaczone zostały wartości zmiennej objaśnianej, czyli sumaryczne długość ścieżek rowerowych w heksagonie.

Struktura GeoDataFrame'u po dodaniu zmiennej objaśnianej:

!['Zmienna objaśniana'](./images/y.png)


**3.2 Zmienne objaśniające**

Kolejnym zadaniem było wyznaczenie zmiennych objaśniających, z pomocą których model miał przewidywać długość ścieżek rowerowych w heksagonach. Do tego celu wykorzystana została biblioteka osmnx, która pozwala w efektywny sposób pozyskiwać dane różnych kategorii z Open Street Map. 

Po konsultacjach i analizie zdecydowano się na ekstrakcję następujących cech:
- długość głównych dróg
- długość ścieżek i chodników
- odległość od centrum miasta
- ilość budynków
- powierzchnia przestrzeni zielonych
- ilość punktów usługowych

Agregacja odbywała się oczywiście względem heksagonu.

### 4. Eksploracyjna analiza danych

Przed przystąpieniem do tworzenia modeli uczenia maszynowego oraz sieci neuronowych, zapoznano się z danymi zapisanymi w ramkach danych. W pierwszej kolejności porównane zostały wszystkie zmienne dla Amsterdamu oraz Krakowa.

!['Porównanie zmiennych'](./images/variables_comparison.png)

Ukazują one sporą dysproporcję w gęstości dróg rowerowych w obydwu miastach. Obydwa miasta wydają się mieć podobną gęstość dróg oraz punktów serwisowych wewnątrz heksagonu. Kraków posiada większą gęstość chodników. Reszta zmiennych również układa się w podobny sposób w obydwu miastach.

Następnie przygotowana została macierz korelacji zmiennych dotyczących Amsterdamu.

!['Correlation matrix'](./images/corr_matrix.png)

Widoczna jest wysoka korelacja długości ścieżek rowerowych z długością ulic oraz chodników. Oznacza to, że większość ścieżek rowerowych budowana jest w ciągu ulic, obok chodnika. Potwierdza to również wysoka korelacja między tymi dwoma zmiennymi. Wysoko skorelowana jest również długość chodników oraz ilość budynków - jest to dość logiczne, gdyż zazwyczaj przy budynkach tworzone są chodniki, aby wygodnie można było się dostać do nich pieszo.

### 5. Przygotowanie modeli

**5.1 Podział na zbiór testowy i treningowy**

Ramka danych została podzielona na zbiór treningowy oraz testowy w proporcji 80:20. Dodatkowo, aby zwiększyć jakość modeli wszystkie zmienne zostały znormalizowane przy użyciu StandardScaler'a z pakietu sklearn.

**5.2 Przygotowanie pierwszych modeli**

Kolejnym krokiem było przygotowanie modeli uczenia maszynowego w celu predykcji długości ścieżek rowerowych w heksagonie, na podstawie wcześniej wyekstrahowanych cech. Zostały przygotowane następujące typy modeli:
- regresja liniowa,
- regresor wektorów nośnych SVR,
- las losowy, 
- GBM,
- oraz sieć neuronowa.

Dla każdego typu został przygotowany wstępny model z podstawowymi hiperparametrami. Poniżej przedstawione są wstępne metryki dla każdego z nich.

| Metric    | Regresja liniowa | SVR | Random Forest | GBM | Neural Network |
|-----|------|------|------|------|------|
| MSE  |  565011.18  |  1348908.69  |  526475.71  |  563273.31  |  584717.17  |
| RMSE  |  751.67  |  1161.42  |  725.58  |  750.51  |  764.66  |
| MAE  |  556.27  |  899.97  |  522.75  |  557.15  |  579.04  |
| R^2  |  0.56  |  -0.05  |  0.59  |  0.56  |  0.54  |

Na podstawie powyżej tabeli można określić, że najlepiej z predykcją porawdził sobie regresor lasu losowego, najgorzej zaś regresor wektorów nośnych SVR. Użycie sieci neuronowej nie sprawiło jakiegokolwiek polepszenia się predykcji względem klasycznych algorymtów ML, z tego też powodu nie będzie ona brana pod uwagę przy treningu hiperparametrów, gdyż jest ona dużo bardziej wymagająca obliczeniowo niż pozostałe modele.

**5.3 Trening hiperparametrów**

Dla czterech pozostałych modeli został przeprowadzony trening hiperparametrów w celu dobrania odpowiedniego modelu do predykcji długości ścieżek rowerowych w heksagonie. Trening został przeprowadzony przy użyciu metody przeszukiwania siatki - Grid Search. Dla poszczególnych modeli zostały przygotowane następujące parametry.

!['Params'](./images/param_grids.png)

Następnie zostało przeprowadzone przeszukiwanie siatki dla każdego modelu.

Najlepsze parametry wraz z metrykami prezentowały się w następujący sposób:
- regresja liniowa
   * najlepsze parametry: {'fit_intercept': True}
   * RMSE: 751.67
   * R2 score: 0.56
- SVR
   * najlepsze parametry: {'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}
   * RMSE: 752.25
   * R2 score: 0.56
- las losowy
   * najlepsze parametry: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
   * RMSE: 738.24
   * R2 score: 0.58
- GBM
   * najlepsze parametry: {'fit_intercept': True}
   * RMSE: 739.98
   * R2 score: 0.57
   
Podobnie jak przed treningiem, najlepszym modelem okazał się las losowy, jednak co warto zauważyć, lepsze metryki miał model pierwotny, z domyślnym parametrem n_estimators=100. Z tego też powodu model ten będzie użyty do predykcji długości ścieżek rowerowych na siatce Krakowa.

### 6. Wykorzystanie najlepszego modelu na siatce dla Krakowa

Ramka z danymi dotyczącymi Krakowa została podzielona na ramkę z wartościami objaśniającymi i tablicę zawierającą realne długości ścieżek rowerowych. Wartości objaśniające zostały również znormalizowane, aby były gotowe do użycia w modelu.

Tak przygotowane dane posłużyły do predykcji z użyciem modelu lasu losowego. Dla predykowanych wartości zostały policzone metryki w porównaniu do wartości rzeczywistych.

- Mean Squared Error (MSE): 1407237.669477436
- Root Mean Squared Error (RMSE): 1186.2704874848046
- Mean Absolute Error (MAE): 963.1584295881288
- R^2 Score: -2.9637195872996003
 
Został przygotowany również wykres porównujący wartości rzeczywiste wraz z predykowanymi.
 
!['Comparison'](./images/prediction_compare.png)
 
 
### 7. Podsumowanie i wnioski
 
W projekcie tym został przeprowadzony proces przygotowania i ekstrakcji danych wejściowych, na podstawie których został przeprowadzony proces stworzenia siatki heksagonalnej dla miasta oraz ekstrakcji różnych cech wykorzystywanych w modelach predykcyjnych.

Następnie przetestowanych zostało pięć różnych modeli, aby sprawdzić który z nich najlepiej predykowałby długość ścieżki w heksagonie dla Amsterdamu. Najlepszym z nich okazał się domyślny regresor Lasu Losowego, który został wybrany do predykcji danych dla Krakowa.

Po użyciu danych z Krakowa, widać wyraźnie że model ten nie jest odpowiedni dla tego miasta. Potwierdzają to metryki (wartość R^2 poniżej zera oznacza że model bardzo słabo wyjaśnia dane), jak i wizualizacja, dzięki której widzimy, że predykowana gęstość ścieżek jest znacznie wyższa niż w rzeczywistości. 

Na taki stan rzeczy ma wpływ kilka czynników:

1. Różnice w infrastrukturze miejskiej - Amsterdam znany jest z rozbudowanej infrastruktury rowerowej, płaskiego terenu i szerokich ulic. Miasto jest zaplanowane i dostosowane do ruchu rowerowego, co wpływa na większą gęstość i długość ścieżek rowerowych. Kraków ma bardziej zróżnicowaną topografię i historyczną zabudowę, która może ograniczać możliwości rozbudowy sieci ścieżek rowerowych. Ulice mogą być węższe i mniej dostosowane do ruchu rowerowego.

2. Kulturowe podejście do transportu - W Holandii rower jest jednym z głównych środków transportu, co powoduje, że miasto inwestuje więcej w rozwój infrastruktury rowerowej. W Polsce mimo wzrostu popularności przemieszczania się rowerem, samochody i transport publiczny są nadal głównymi środkami transportu, co może wpływać na mniejszy priorytet dla rozbudowy ścieżek rowerowych.

3. Warunki geograficzne i klimatyczne - Amsterdam posiada klimat umiarkowany morski, łagodniejsze zimy i brak znacznych wzniesień, co sprzyja całorocznemu korzystaniu z rowerów. Kraków zaś znajduje się w klimacie kontynentalnym, w którym zimy mogą być surowsze, a teren bardziej zróżnicowany pod względem wysokości, co może ograniczać sezon rowerowy i wpływać na decyzje dotyczące rozbudowy ścieżek.

4. Różnice historyczne i urbanistyczne - Amsterdam to miasto o strukturze bardziej dostosowanej do nowoczesnego planowania transportu. Kraków posiada historyczną zabudowę i wąskie uliczki, które mogą stanowić wyzwanie w rozbudowie infrastruktury rowerowej.

Z tego też względu, model predykcyjny stworzony dla Amsterdamu nie uwzględnia wielu specyficznych czynników charakterystycznych dla Krakowa. W związku z tym, aby prognozowanie długości ścieżek rowerowych w Krakowie było dokładne i użyteczne, należy stworzyć model uwzględniający lokalne warunki i kontekst. Model ten mógłby być dobry dla innych miast holenderskich posiadających podobne cechy kulturowo-urbanistyczne jak Amsterdam.